## socket 最底层的API

socket是以标准库的形式来提供的

socket跟网络是不一样的

socket工作在第几层，并不严谨。
大多数时候，socket控制的是传输层 TCP/UDP，IP（也可以）

socket 是一个编程接口，并不一定需要TCP/IP网络，也不工作在具体的某一层

socket是操作系统提供的编程接口

In [1]:
import socket

* socket.AF_INET  ## ipv4
* socket.AF_INET6 ## ipv6
* socket.AF_UNIX  ## unix domain socket   

支持的很多，主要用的多的这几种

* socket.SOCK_DGRAM  ## UDP  
* socket.SOCK_STREAM ## TCP       TCP UDP 是由操作系统内核来封包的 都是标准的包
* socket.SOCK_RAW  ## 这不是一个协议 即不是UDP 也不是TCP，它需要用户空间自己封包 如:icmp/igmp/网络攻击

常用的就些


In [ ]:
主要讲 TCP/UDP

### TCP server/client

In [3]:
## help(socket.socket)     它是一个类，继承自_socket.socket   下划线开始的，惯例都是C语言实现的

In [5]:
so = socket.socket()    ## 参数保持默认，就创建了一个tcp的ipv4的 socket 实例     初始化一个socket实例

In [6]:
help(so.bind)

Help on built-in function bind:

bind(...) method of socket.socket instance
    bind(address)
    
    Bind the socket to a local address.  For IP sockets, the address is a
    pair (host, port); the host must refer to the local host. For raw packet
    sockets the address is a tuple (ifname, proto [,pkttype [,hatype]])



In [7]:
so.bind(('127.0.0.1',1234))   ## 绑定 地址+端口

ss -tanl   并没有发现 1234 这个端口，为什么呢？因为还没有监听

In [9]:
so.listen()        ## 监听

再 ss -tanl 就有了

监听之后，就可以telnet连接上它了，telnet 连接后，就可以操作了

In [10]:
help(so.accept)

Help on method accept in module socket:

accept() method of socket.socket instance
    accept() -> (socket object, address info)
    
    Wait for an incoming connection.  Return a new socket
    representing the connection, and the address of the client.
    For IP sockets, the address info is a pair (hostaddr, port).



In [12]:
s, info = so.accept()  
## 这个应该返回的，可以不返回呢，阻塞了！   ## accept 返回一个socket实例和客户端地址

KeyboardInterrupt: 

In [13]:
help(s.recv)

Help on built-in function recv:

recv(...) method of socket.socket instance
    recv(buffersize[, flags]) -> data
    
    Receive up to buffersize bytes from the socket.  For the optional flags
    argument, see the Unix manual.  When no data is available, block until
    at least one byte is available or until the remote end is closed.  When
    the remote end is closed and all data is read, return the empty string.



In [16]:
s.recv(1024)    ## 看到，跟telnet 里敲的一样   ## accept返回的socket实例，可以接受数据

b'sdfjlk\r\nfds\r\n1234\r\nsdjf\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nsfd\r\nfds\r\n'

In [17]:
info      ##  客户端地址     55824 是客户端的一个端口   

('127.0.0.1', 55824)

In [18]:
s   ## socket 对象

<socket.socket fd=49, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 55824)>

In [ ]:
[root@python3 ~]# netstat -anpl | grep 1234
tcp        0      0 127.0.0.1:1234          0.0.0.0:*               LISTEN      14215/python3.5     
tcp        0      0 127.0.0.1:55824         127.0.0.1:1234          ESTABLISHED 16462/telnet        
tcp        6      0 127.0.0.1:1234          127.0.0.1:55824         ESTABLISHED 14215/python3.5     
[root@python3 ~]#    一个出站连接，一个入站连接

In [19]:
so   ## 也是socket 对象

<socket.socket fd=50, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234)>

In [20]:
s.recv(1024)

b'\r\n\r\n\r\nabcd\r\n'

In [21]:
s.recv(1024)

b'dddd\r\n'

In [22]:
s.recv(1024)

b'ccie\r\n'

socket服务端编程，就这几步

* 1.初始化socket实例
* 2.bind
* 3.listen
* 4.accept   到这里服务端就OK了
* 5.recv     这一步服务端并不需要

前四个步骤一个也不能少！

这里只接收数据，为什么不返回数据？ 
返回一个看看

In [23]:
s.send(b'123456')    ## 果然在telnet界面接收到返回了    send 是发送bytes  # accept 返回的socket实例，可以发送数据

6

In [24]:
s.close()    ## 关闭客户端的连接

In [ ]:
## 可以看到telnet客户端已经关闭

Connection closed by foreign host.
(python) [root@python3 notes]# 

In [ ]:
## 服务端并没有关闭

[root@python3 ~]# netstat -anpl | grep 1234
tcp        0      0 127.0.0.1:1234          0.0.0.0:*               LISTEN      14215/python3.5 

In [25]:
## 再来看

s, _ = so.accept()        ## 看，并不返回 阻塞了    
                         ## accept 是阻塞的，直到客户端telnet建立好连接了 就不阻塞了

In [26]:
s

<socket.socket fd=49, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 55825)>

In [ ]:
[root@python3 ~]# netstat -anpl | grep 1234
tcp        0      0 127.0.0.1:1234          0.0.0.0:*               LISTEN      14215/python3.5     
tcp        0      0 127.0.0.1:55825         127.0.0.1:1234          ESTABLISHED 18641/telnet        
tcp        0      0 127.0.0.1:1234          127.0.0.1:55825         ESTABLISHED 14215/python3.5     
[root@python3 ~]# 

accept是返回连接的信息的，获取连接信息的

每个socket其实需要对应一个文件描述符，listen需要一个文件描述符，每个连接，也对应一个文件描述符

In [ ]:
再开一个telnet 客户端，然后

In [27]:
s2, _ = so.accept()

In [28]:
s2

<socket.socket fd=51, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 55826)>

In [30]:
s.send(b'1234\n')   ## 这个，发向第一个连接上

5

In [31]:
s2.send(b'abcd\n')   ## 而这个，是发向第二个连接上

5

In [32]:
s2.close()

In [33]:
s.close()      ## 都关掉了

怎么把监听地址也关掉呢？

In [34]:
so.close()

In [ ]:
[root@python3 ~]# netstat -anpl | grep 1234

一个实际的例子：
写一个聊天服务器

聊天室这种，每条消息都是广播

In [3]:
import socket

In [2]:
import threading    ## 因为需要并发，所以还要引入 threading

唉！

除了recv和send，还有其它发送数据的方法

In [4]:
so = socket.socket()

In [5]:
so.bind(('127.0.0.1',10086))

In [6]:
so.listen()

In [7]:
s,_ =  so.accept()

In [8]:
s

<socket.socket fd=50, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 10086), raddr=('127.0.0.1', 33110)>

In [9]:
help(s.recv)

Help on built-in function recv:

recv(...) method of socket.socket instance
    recv(buffersize[, flags]) -> data
    
    Receive up to buffersize bytes from the socket.  For the optional flags
    argument, see the Unix manual.  When no data is available, block until
    at least one byte is available or until the remote end is closed.  When
    the remote end is closed and all data is read, return the empty string.



In [10]:
help(s.recv_into)

Help on built-in function recv_into:

recv_into(...) method of socket.socket instance
    recv_into(buffer, [nbytes[, flags]]) -> nbytes_read
    
    A version of recv() that stores its data into a buffer rather than creating 
    a new string.  Receive up to buffersize bytes from the socket.  If buffersize 
    is not specified (or 0), receive up to the size available in the given buffer.
    
    See recv() for documentation about the flags.



In [11]:
help(s.recvfrom)

Help on built-in function recvfrom:

recvfrom(...) method of socket.socket instance
    recvfrom(buffersize[, flags]) -> (data, address info)
    
    Like recv(buffersize, flags) but also return the sender's address info.



In [12]:
help(s.send)   ## 会发送数据到缓冲区中

Help on built-in function send:

send(...) method of socket.socket instance
    send(data[, flags]) -> count
    
    Send a data string to the socket.  For the optional flags
    argument, see the Unix manual.  Return the number of bytes
    sent; this may be less than len(data) if the network is busy.



In [13]:
help(s.sendall)

Help on built-in function sendall:

sendall(...) method of socket.socket instance
    sendall(data[, flags])
    
    Send a data string to the socket.  For the optional flags
    argument, see the Unix manual.  This calls send() repeatedly
    until all data is sent.  If an error occurs, it's impossible
    to tell how much data has been sent.



In [14]:
f = s.makefile()   ## 这个方法才是python 的杀器

In [15]:
help(s.makefile)   ## 它把socket对象转化成类文件对象

Help on method makefile in module socket:

makefile(mode='r', buffering=None, *, encoding=None, errors=None, newline=None) method of socket.socket instance
    makefile(...) -> an I/O stream connected to the socket
    
    The arguments are as for io.open() after the filename, except the only
    supported mode values are 'r' (default), 'w' and 'b'.



In [17]:
f.read(5)            ##  阻塞了    在telnet里输入了12345共5个字符，它就返回了

'12345'

In [18]:
f.readline()

'\n'

In [19]:
f.readline()          ## 阻塞了    telnet里输入1234回车    它会一行一行的读入

'1234\n'

read是读到缓冲区，readline是按行来读的

In [20]:
w = s.makefile('w')

In [21]:
help(w.write)

Help on built-in function write:

write(text, /) method of _io.TextIOWrapper instance
    Write string to stream.
    Returns the number of characters written (which is always equal to
    the length of the string).



In [22]:
w.write('1234567\n')

8

In [23]:
w.flush()   ## flush 后，数据就过去了

这里无论是read还是write，都是用的字符串，而recv和send 都是bytes.

In [24]:
b = s.makefile('br')       ## 让它为bytes也是可以的

In [25]:
b.readline()    ## 阻塞了   telnet 里输入abcd 回车

b'abcd\r\n'

In [26]:
s.close()

In [27]:
b.closed

False

In [28]:
b.readable()

True

In [29]:
f.close()

In [30]:
w.close()

In [31]:
b.close()

** makefile 生成的类文件对象全部close之后，socket才能真正close **

In [ ]:
## 上句非常重要！！

In [32]:
so.getsockname()     ## 获取本地地址

('127.0.0.1', 10086)

In [33]:
so.getpeername()     ## 获取对端地址

OSError: [Errno 107] Transport endpoint is not connected

In [34]:
help(so.settimeout)    ## 用来设置超时时间的

Help on built-in function settimeout:

settimeout(...) method of socket.socket instance
    settimeout(timeout)
    
    Set a timeout on socket operations.  'timeout' can be a float,
    giving in seconds, or None.  Setting a timeout of None disables
    the timeout feature and is equivalent to setblocking(1).
    Setting a timeout of zero is the same as setblocking(0).



写服务器的话，通常不会上socket模块的。

In [1]:
import socketserver

socketserver 这个就是标准库提供的用来写我们server端程序的框架

写服务端的话，基本都是socketserver